# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [2]:
file = "../output_data/cities"
cities_df = pd.read_csv(file)

cities_df.describe()

,City ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,5.720000e+02
mean,285.500000,20.141194,15.073134,22.227552,66.664336,51.138112,3.577080,1.627750e+09
std,165.266452,32.498647,92.454052,8.180766,22.593939,37.698595,2.392447,2.410350e+02
min,0.000000,-54.800000,-179.166700,0.270000,8.000000,0.000000,0.000000,1.627749e+09
25%,142.750000,-6.071225,-69.015800,16.235000,54.000000,12.000000,1.917500,1.627750e+09
50%,285.500000,23.063400,19.567250,23.210000,72.000000,50.000000,3.080000,1.627750e+09
75%,428.250000,46.518350,102.334775,27.560000,84.000000,90.000000,5.002500,1.627750e+09
max,571.000000,78.218600,178.416700,45.240000,100.000000,100.000000,13.380000,1.627750e+09


### Humidity Heatmap

In [3]:
gmaps.configure(api_key=g_key)

In [11]:
#generate list of humidity
list_humidity = cities_df["Humidity"].tolist()

#establish google map
fig = gmaps.figure()
#provide lat/long
locations = cities_df[["Lat", "Lng"]]

#state heatmap required, weight based on humidity
heat_layer = gmaps.heatmap_layer(locations, weights=list_humidity)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## DataFrame of Ideal Weather Conditions

In [5]:
low_wind_df = len(cities_df[cities_df["Wind Speed"] <= 10])
good_temp = len(cities_df[(cities_df["Max Temp"] > 15) & (cities_df["Max Temp"] < 30)])
zero_clouds = len(cities_df[cities_df["Cloudiness"] == 0])

ideal_criteria = cities_df[(cities_df["Wind Speed"] <= 10) &
                           (cities_df["Max Temp"] > 15) &
                           (cities_df["Max Temp"] < 30) &
                           (cities_df["Cloudiness"] == 0)]

ideal_criteria.head(20).reset_index(drop=True)

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,1,Alice Springs,-23.7000,133.8833,17.75,32,0,3.09,AU,1627749576
1,18,Cape Town,-33.9258,18.4232,18.48,82,0,2.06,ZA,1627748931
2,35,Ribeira Brava,32.6500,-17.0667,25.35,65,0,3.08,PT,1627749549
3,53,Saskylakh,71.9167,114.0833,19.24,38,0,3.99,RU,1627749641
4,63,East London,-33.0153,27.9116,20.52,37,0,3.09,ZA,1627749643
5,70,Aranos,-24.1333,19.1167,15.78,23,0,5.20,NaN,1627749643
6,73,Port Elizabeth,-33.9180,25.5701,17.83,59,0,2.57,ZA,1627749644
7,94,Seoul,37.5683,126.9778,26.78,78,0,1.54,KR,1627749671
8,115,Ponta do Sol,32.6667,-17.1000,25.35,65,0,2.57,PT,1627749708
9,165,Lüderitz,-26.6481,15.1594,21.56,32,0,4.07,NaN,1627749774


### Hotel Map

In [6]:
list_hotels = []
list_cities = []
list_countries = []
list_lat = []
list_lng = []

p = 0
for index, city in ideal_criteria.iterrows():
        
        #set lat and long or search
        lat = city["Lat"]
        lng = city["Lng"]
        print(lat,lng)
        #create params for search
        target_coordinates = f"{lat}, {lng}"
        target_search = "Hotel"
        target_radius = 5000
        target_type = "lodging"
        
        params = {
            "location": target_coordinates,
            "keyword": target_search,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
                }
        
        #request json for search results
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params=params)
        hotel_data = response.json()
        try:
            #gather info from json
            list_hotels.append(hotel_data["results"][0]["name"])
            list_cities.append(hotel_data["results"][0]["vicinity"])
            list_countries.append(city["Country"])
            list_lat.append(lat)
            list_lng.append(lng)
            p = p + 1
            print(f"Result {p} found, appending")
        except (KeyError, IndexError):
            print("Error, going to next")
        if p == 10:
            break
print("-----------------------------\nData Retrieval Complete\n-----------------------------")

-23.7 133.8833
Result 1 found, appending
-33.9258 18.4232
Result 2 found, appending
32.65 -17.0667
Result 3 found, appending
71.9167 114.0833
Error, going to next
-33.0153 27.9116
Result 4 found, appending
-24.1333 19.1167
Result 5 found, appending
-33.918 25.5701
Result 6 found, appending
37.5683 126.9778
Result 7 found, appending
32.6667 -17.1
Result 8 found, appending
-26.6481 15.1594
Result 9 found, appending
43.3027 77.2395
Result 10 found, appending
-----------------------------
Data Retrieval Complete
-----------------------------


In [8]:
hotel_df = pd.DataFrame(
    {"Hotel Name" : list_hotels,
            "City" : list_cities,
            "Country" : list_countries,
            "Lat" : list_lat,
            "Lng" : list_lng})
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].str.title()

hotel_df

,Hotel Name,City,Country,Lat,Lng
0,"Crowne Plaza Alice Springs Lasseters, An Ihg H...","93 Barrett Dr, Alice Springs",AU,-23.7000,133.8833
1,The Silo Hotel,"Silo Square, Victoria & Alfred Waterfront, Cap...",ZA,-33.9258,18.4232
2,Estalagem Da Ponta Do Sol,"Caminho do Passo, Ponta do Sol",PT,32.6500,-17.0667
3,Southern Sun Hemingways,"Western Ave, Ocean View, East London",ZA,-33.0153,27.9116
4,Ley-Lia Guest House,"6 Sarel Theron Street, Aranos",NaN,-24.1333,19.1167
5,Island Way Villa,"2 Hartley Rd, Millard Grange, Gqeberha",ZA,-33.9180,25.5701
6,Banyan Tree Club & Spa Seoul,"60 Jangchungdan-ro, Jangchung-dong, Jung-gu",KR,37.5683,126.9778
7,Estalagem Da Ponta Do Sol,"Caminho do Passo, Ponta do Sol",PT,32.6667,-17.1000
8,Lüderitz Nest Hotel,"820 Dias St, Luderitz",NaN,-26.6481,15.1594
9,Eco Guesthouse,"Алатау 2А, Алмалык, Almaty",KZ,43.3027,77.2395


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_pins = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
rows_lat_long = hotel_df[["Lat", "Lng"]]
# NOTE: be sure to update with your DataFrame name

In [10]:
# create pins/markers
hotel_markers = gmaps.marker_layer(
    rows_lat_long,info_box_content=hotel_pins
)

# Add pins onto heatmap
fig.add_layer(hotel_markers)
fig

Figure(layout=FigureLayout(height='420px'))